# Bayesian Linear Regression VI (using BNN wrapper) vs closed-form solution

In [139]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [140]:
%pip install -q torch

Note: you may need to restart the kernel to use updated packages.


In [141]:
import numpy as np

import torch

In [142]:
torch.__version__

'1.13.0+cu117'

In [143]:
import sys
sys.path.append("../")

In [144]:
import reparameterized as r
from reparameterized import sampling
from reparameterized import likelihoods
from reparameterized import densities
from reparameterized import bnn_wrapper

In [145]:
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (5, 5) # Width and height

In [146]:
if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    env = torch.cuda
    print("Using GPU")
else:
    torch.set_default_tensor_type('torch.FloatTensor')
    env = torch
    print("Using CPU")

Using CPU


In [147]:
SEED = 123

torch.manual_seed(SEED)
np.random.seed(SEED)

## Data

In [148]:
# # Toy Data

# def generate_toy_data(N):
#     x_data = np.float32(np.random.uniform(-10.5, 10.5, (1, N))).T
#     r_data = np.float32(np.random.normal(size=(N,1)))
#     y_data = np.float32(np.sin(0.75*x_data)*7.0+x_data*0.5+r_data*2.0)+10
    
#     mask = (x_data<0.0) | (x_data>5.0)
#     r_data = r_data[mask]
#     y_data = y_data[mask, None]
#     x_data = x_data[mask, None]

#     return x_data, y_data, r_data


# x_data, y_data, r_data = generate_toy_data(100)

In [149]:
# Boston dataset

import pandas as pd

data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

x_data = data
y_data = target[:,None]


In [150]:
x = torch.tensor(x_data, dtype = torch.float, requires_grad=False)
y = torch.tensor(y_data, dtype = torch.float, requires_grad=False)

print("x=%s y=%s" % (x.shape, y.shape))

x=torch.Size([506, 13]) y=torch.Size([506, 1])


## Network

In [151]:
INPUT_DIM = x_data.shape[-1]
OUTPUT_DIM = y_data.shape[-1]

# Linear regression model
model = torch.nn.Sequential(
    torch.nn.Linear(INPUT_DIM, OUTPUT_DIM),
    )    
 

List model parameters:

In [152]:
for k, p in model.named_parameters():
    print(f"{k}: {p.shape}")

0.weight: torch.Size([1, 13])
0.bias: torch.Size([1])


## Wrapper

In [153]:
bnn = bnn_wrapper.BayesianNeuralNetwork(model)

## Priors

In [154]:
bnn.set_prior_densities(densities.create_gaussian_nll)

## Likelihood

In [155]:
LIKELIHOOD_SIGMA = 100

def predictive_distribution_sampler(logits, **kwargs): 
    return likelihoods.factorized_gaussian_with_fixed_scale_sample(logits, scale=LIKELIHOOD_SIGMA, **kwargs)

def predictive_distribution_log_lik(logits, output_y, **kwargs): 
    return likelihoods.factorized_gaussian_with_fixed_scale_log_prob(logits, output_y, scale=LIKELIHOOD_SIGMA, **kwargs)

In [156]:
bnn.predictive_distribution_sampler = predictive_distribution_sampler
bnn.predictive_distribution_log_lik = predictive_distribution_log_lik

## Create samplers for parameters

In [157]:
# # factorized gaussian
# sampler, variational_params1, aux_objs = sampling.create_multiparameter_sampler(sampling.create_factorized_gaussian_sampler, model.named_parameters())

# normalizing flow
from reparameterized.sampling.realnvp import build_realnvp
sampler, variational_params1, aux_objs = sampling.create_multiparameter_sampler(
    sampling.create_flow_sampler, 
    model.named_parameters(),
    build_flow_func=build_realnvp,
    realnvp_m=8,  
    realnvp_num_layers=16,
)

**Note, we use only one sampler returning joint samples for all parameters.**

In [158]:
parameter_names = tuple(n for n, _ in model.named_parameters())

bnn.set_posterior_sampler(parameter_names, sampler, variational_params1)

# Preview sample

In [159]:
print("samplers:")
display(bnn.parameters2sampler)

samplers:


{('0.weight',
  '0.bias'): <function reparameterized.sampling.multiparameter._multiparameter_sampler_unpack.<locals>.wrapped_sampler(n_samples=1)>}

In [160]:
print("variational parameters:")
display([(vn, vp.shape) for vn, vp in bnn.variational_params])

variational parameters:


[('0.weight_0.bias:t.0.0.weight', torch.Size([8, 7])),
 ('0.weight_0.bias:t.0.0.bias', torch.Size([8])),
 ('0.weight_0.bias:t.0.2.weight', torch.Size([8, 8])),
 ('0.weight_0.bias:t.0.2.bias', torch.Size([8])),
 ('0.weight_0.bias:t.0.4.weight', torch.Size([7, 8])),
 ('0.weight_0.bias:t.0.4.bias', torch.Size([7])),
 ('0.weight_0.bias:t.1.0.weight', torch.Size([8, 7])),
 ('0.weight_0.bias:t.1.0.bias', torch.Size([8])),
 ('0.weight_0.bias:t.1.2.weight', torch.Size([8, 8])),
 ('0.weight_0.bias:t.1.2.bias', torch.Size([8])),
 ('0.weight_0.bias:t.1.4.weight', torch.Size([7, 8])),
 ('0.weight_0.bias:t.1.4.bias', torch.Size([7])),
 ('0.weight_0.bias:t.2.0.weight', torch.Size([8, 7])),
 ('0.weight_0.bias:t.2.0.bias', torch.Size([8])),
 ('0.weight_0.bias:t.2.2.weight', torch.Size([8, 8])),
 ('0.weight_0.bias:t.2.2.bias', torch.Size([8])),
 ('0.weight_0.bias:t.2.4.weight', torch.Size([7, 8])),
 ('0.weight_0.bias:t.2.4.bias', torch.Size([7])),
 ('0.weight_0.bias:t.3.0.weight', torch.Size([8, 7])),


In [161]:
parameters_samples, nlls = bnn.sample_posterior(n_samples=3)

In [162]:
print("parameters' sample:")
{n: s.shape for n, s in parameters_samples.items()}

parameters' sample:


{'0.weight': torch.Size([3, 1, 13]), '0.bias': torch.Size([3, 1])}

In [163]:
print("parameters' NLLs:")
nlls

parameters' NLLs:


tensor([[14.1378, 15.3474, 16.2944]], grad_fn=<StackBackward0>)

In [164]:
bnn.sample_predictive(x, n_samples=2, n_predictive_samples=3, flatten_samples_dims=False).shape

torch.Size([3, 2, 506, 1])

In [165]:
bnn.predictive_likelihoods(x, y).shape

torch.Size([1, 506])

## Learning

In [166]:
optimized_parameters = [vp for _, vp in bnn.variational_params]
optimizer = torch.optim.Adam(optimized_parameters, lr=0.001) 

n_posterior_samples = 10
n_epochs = 10000

for e in range(n_epochs):
    optimizer.zero_grad()

    # sampling from approximate posterior
    parameters_samples, posterior_nlls = bnn.sample_posterior(n_samples=n_posterior_samples)
    log_q = -posterior_nlls.sum(0)  # sum over parameters

    # priors
    prior_nlls = bnn.prior_nll(parameters_samples)
    log_p = -prior_nlls.sum(0)  # sum over parameters

    # estimating likelihoods of observed y
    logliks = bnn.predictive_likelihoods(x, y, parameters_samples)
    assert logliks.shape==torch.Size([n_posterior_samples, len(y)])
    logliks = logliks.sum(1)  # sum over data points
    
    elbo = logliks+log_p-log_q
    elbo = elbo.mean()  # average over posterior samples

    loss_vi = -elbo
    loss_vi.backward()
    optimizer.step()

    if e<10 or e%100==0:
        print(f"epoch={e}: elbo = {elbo: .2f} (= {logliks.mean(): .2f} [+] {log_p.mean(): .2f} [-] {log_q.mean(): .2f})")

epoch=0: elbo = -5609.82 (= -5597.69 [+] -29.14 [-] -17.00)
epoch=1: elbo = -5269.90 (= -5257.65 [+] -29.09 [-] -16.85)
epoch=2: elbo = -6665.23 (= -6655.85 [+] -25.66 [-] -16.28)
epoch=3: elbo = -5580.00 (= -5570.40 [+] -25.55 [-] -15.95)
epoch=4: elbo = -4508.60 (= -4500.56 [+] -25.75 [-] -17.71)
epoch=5: elbo = -4784.37 (= -4776.29 [+] -25.63 [-] -17.55)
epoch=6: elbo = -4007.08 (= -3995.75 [+] -28.47 [-] -17.14)
epoch=7: elbo = -4406.60 (= -4392.26 [+] -30.57 [-] -16.23)
epoch=8: elbo = -5151.36 (= -5141.00 [+] -28.38 [-] -18.02)
epoch=9: elbo = -4350.73 (= -4337.38 [+] -31.12 [-] -17.77)
epoch=100: elbo = -2876.93 (= -2858.56 [+] -27.04 [-] -8.68)
epoch=200: elbo = -2859.32 (= -2839.10 [+] -26.13 [-] -5.91)
epoch=300: elbo = -2849.90 (= -2831.71 [+] -23.00 [-] -4.81)
epoch=400: elbo = -2828.23 (= -2806.60 [+] -21.69 [-] -0.06)
epoch=500: elbo = -2835.41 (= -2819.28 [+] -18.00 [-] -1.88)
epoch=600: elbo = -2820.89 (= -2806.14 [+] -18.08 [-] -3.33)
epoch=700: elbo = -2826.61 (= -281

# Linear Regression Bayesian true (closed-form) posterior

See Bishop's book

In [167]:
psi = np.hstack([x_data, np.ones(x_data.shape)])
t = y_data

In [168]:
psi.shape, t.shape

((506, 26), (506, 1))

In [169]:
#likelihood precision:
beta = 1./(LIKELIHOOD_SIGMA*LIKELIHOOD_SIGMA)

#prior:
S0 = np.eye(psi.shape[-1])
m0 = np.zeros(psi.shape[-1]).reshape(-1, 1)

In [170]:
SN = np.linalg.inv( np.linalg.inv(S0) + beta*np.dot(psi.T, psi) ) # covariance matrix (Bishop 3.50)

In [171]:
mN = np.dot(SN, np.dot(np.linalg.inv(S0), m0) + beta*np.dot(psi.T, t)) # means (Bishop 3.51)

Compare expectations (means):

In [172]:
dim = SN.shape[0]//2+1

mN[:dim].flatten().round(2)

array([-0.09,  0.12, -0.06,  0.02,  0.01,  0.28,  0.09,  0.02,  0.03,
        0.  ,  0.18,  0.04, -0.47,  0.02])

In [173]:
# empirically estimate means for the VI solution:
parameters_samples, nlls = bnn.sample_posterior(n_samples=10000)
torch.round(parameters_samples['0.weight'].mean(dim=0).flatten(), decimals=2), parameters_samples['0.bias'].mean()

(tensor([-0.1200,  0.0900, -0.0600,  0.0100,  0.1500,  0.3400,  0.0600,  0.2300,
          0.0600,  0.0000,  0.1000,  0.0500, -0.4800], grad_fn=<RoundBackward1>),
 tensor(-0.0092, grad_fn=<MeanBackward0>))

Compare covariance:

In [174]:
dim = SN.shape[0]//2+1

SN = SN[:dim, :dim]

SN.round(3)

array([[ 0.306, -0.003,  0.015,  0.001,  0.   ,  0.005, -0.003,  0.013,
        -0.065, -0.004,  0.005,  0.004, -0.04 ,  0.001],
       [-0.003,  0.051,  0.036, -0.   , -0.   , -0.018,  0.014, -0.037,
         0.021, -0.002, -0.001, -0.002,  0.005, -0.002],
       [ 0.015,  0.036,  0.564, -0.003, -0.002,  0.012, -0.027,  0.04 ,
         0.062, -0.014,  0.008,  0.002, -0.044,  0.   ],
       [ 0.001, -0.   , -0.003,  0.997, -0.   , -0.001, -0.001,  0.001,
        -0.002,  0.   ,  0.002, -0.   ,  0.003, -0.   ],
       [ 0.   , -0.   , -0.002, -0.   ,  1.   , -0.002, -0.002, -0.   ,
         0.001, -0.   , -0.003, -0.   , -0.001, -0.   ],
       [ 0.005, -0.018,  0.012, -0.001, -0.002,  0.946, -0.018, -0.026,
         0.014, -0.003, -0.07 , -0.007,  0.028, -0.005],
       [-0.003,  0.014, -0.027, -0.001, -0.002, -0.018,  0.044,  0.018,
         0.01 , -0.003, -0.018, -0.003, -0.041, -0.002],
       [ 0.013, -0.037,  0.04 ,  0.001, -0.   , -0.026,  0.018,  0.908,
         0.022, -0.002, -

In [175]:
# empirically estimate covariance matrix for the VI solution:
parameters_samples, nlls = bnn.sample_posterior(n_samples=10)
samples_weight, samples_bias = parameters_samples['0.weight'].flatten(start_dim=1), parameters_samples['0.bias'].flatten(start_dim=1)
samples = torch.hstack([samples_weight, samples_bias]).T
cov = torch.cov(samples)

In [176]:
# Variances
var1 = np.array([SN[i,i].round(4) for i in range(SN.shape[0])])
var2 = np.array([float(cov[i,i]) for i in range(SN.shape[0])])

var1.round(3), var2.round(3)

(array([0.306, 0.051, 0.564, 0.997, 1.   , 0.946, 0.044, 0.908, 0.615,
        0.003, 0.696, 0.002, 0.424, 0.999]),
 array([6.900e-02, 5.100e-02, 9.380e-01, 2.240e-01, 1.575e+00, 1.535e+00,
        5.400e-02, 9.260e-01, 3.250e-01, 1.000e-03, 3.250e-01, 1.000e-03,
        3.360e-01, 1.385e+00]))